In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# SET UP THE NETWORK
# 1 - NETWORK SIZE:
Ne = 800
Ni = 200

In [ ]:
# 2 - GLOBAL PARAMETERS THAT SET OUR NEURON MODEL. DEFAULT IS SPIKING NEURON:
re = np.random.rand(Ne)
ri = np.random.rand(Ni)
a = np.concatenate((0.02 * np.ones(Ne), 0.02 + 0.08 * ri))
b = np.concatenate((0.2 * np.ones(Ne), 0.25 - 0.05 * ri))
c = np.concatenate((-65 + 15 * re**2, -65 * np.ones(Ni)))
d = np.concatenate((8 - 6 * re**2, 2 * np.ones(Ni)))

In [ ]:
# 3 - SET UP THE CONNECTIVITY MATRIX: DIRECTED NETWORK
frac_delete = 0.6  # Set this fraction of connections to zero.
A = np.random.rand(Ne + Ni, Ne + Ni)
A[A < frac_delete] = 0
A[A > 0] = 1
np.fill_diagonal(A, 0)
plt.figure()
plt.imshow(A)
plt.show()

In [ ]:
# 4 - SET SYNAPTIC WEIGHTS (STRENGTHS) OF CONNECTIONS.
MAX_EXC_WEIGHT = 1
MAX_INH_WEIGHT = 0.5
W = np.concatenate((MAX_EXC_WEIGHT * np.random.rand(Ne + Ni, Ne),
                    -MAX_INH_WEIGHT * np.random.rand(Ne + Ni, Ni)))

In [ ]:

# 5 - The final connectivity matrix S is the element-wise multiplication of A and W.
S = np.multiply(A, W)  # Note that S is directed and weighted!!

In [ ]:
# 6 - DEFINE NOISE STRENGTH. Remember that noise (or random inputs) is the main drive of spontaneous activity.
NOISE_MAX = 0.5

# MAIN SIMULATION
v = -65 * np.ones(Ne + Ni)  # Initial values of v
u = b * v  # Initial values of u
firings = []  # spike timings

for t in range(1000):  # simulation of 1000 ms
    I = np.concatenate((NOISE_MAX * np.random.randn(Ne),
                        2 * np.random.randn(Ni)))  # NOISE or thalamic input
    fired = np.where(v >= 30)[0]  # indices of spikes
    firings.extend(list(zip([t] * len(fired), fired)))
    v[fired] = c[fired]
    u[fired] = u[fired] + d[fired]
    I = I + np.sum(S[:, fired], axis=1)
    v = v + 0.5 * (0.04 * v ** 2 + 5 * v + 140 - u + I)  # step 0.5 ms
    v = v + 0.5 * (0.04 * v ** 2 + 5 * v + 140 - u + I)  # for numerical stability
    u = u + a * (b * v - u)

# PLOT RESULTS
# Raster plot. Time on X, neuron on Y.
firings = np.array(firings)
plt.figure()
plt.scatter(firings[:, 0], firings[:, 1])
plt.xlabel('time (ms)')
plt.ylabel('neuron')
plt.show()